In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Define the URL template and parameters
url_template = "https://www.magicbricks.com/property-for-sale/residential-real-estate?bedroom={bhk}&proptype=Multistorey-Apartment,Builder-Floor-Apartment,Penthouse,Studio-Apartment,Residential-House,Villa&cityName={city}"
bhk_values = [1, 2, 3, 4, 5]
cities = ["Kozhikode", "Kochi", "Trivandrum", "Mumbai", "Bangalore", "Delhi","Kolkata","Gurgaon","Chennai","Hyderabad","Pune"]

# Initialize an empty list to collect all data
data = []

# Loop through each BHK value and each city
for bhk in bhk_values:
    for city in cities:
        # Substitute the BHK and city values into the URL
        url = url_template.format(bhk=bhk, city=city)
        print(f"Fetching data for {bhk} BHK in {city}...")

        try:
            # Send the request
            response = requests.get(url)
            response.raise_for_status()  # Check if the request was successful
            soup = BeautifulSoup(response.text, 'html.parser')

            # Extract property cards
            properties = soup.find_all("div", class_="mb-srp__card")
            if not properties:
                print(f"No properties found for {bhk} BHK in {city}.")
                continue

            for property in properties:
                try:
                    # Extract property details
                    title = property.find("h2", class_="mb-srp__card--title").get_text(strip=True) if property.find("h2", class_="mb-srp__card--title") else "N/A"
                    price = property.find("div", class_="mb-srp__card__price--amount").get_text(strip=True) if property.find("div", class_="mb-srp__card__price--amount") else "N/A"
                    furnishing = property.find("div", {"data-summary": "furnishing"}).find("div", class_="mb-srp__card__summary--value").get_text(strip=True) if property.find("div", {"data-summary": "furnishing"}) else "N/A"
                    link = property.find("script", type="application/ld+json").string
                    link_url = eval(link)['@id'] if link else "N/A"

                    # Append the data to the list without the BHK field
                    data.append({
                        "City": city,
                        "Title": title,
                        "Price": price,
                        "Furnishing": furnishing,
                        "Link": link_url
                    })

                    # Print data to console
                    print(f"{city}: {title} | Price: {price} | Furnishing: {furnishing} | Link: {link_url}")

                except Exception as e:
                    print(f"Error extracting property data for {bhk} BHK in {city}: {e}")

            # Add a delay to avoid too frequent requests
            time.sleep(2)

        except requests.RequestException as e:
            print(f"Error fetching data for {bhk} BHK in {city}: {e}")

# Convert the data to a DataFrame and save it as CSV
if data:
    df = pd.DataFrame(data)
    df.to_csv("Property_data_eleven_cities1.csv", index=False)
    print("Data fetching and saving complete.")
else:
    print("No data collected to save.")


Fetching data for 1 BHK in Kozhikode...
Kozhikode: 1 BHK Flat for Sale in Landmark Village, Kaithappoyil, Kozhikode | Price: ₹41 Lac | Furnishing: Semi-Furnished | Link: https://www.magicbricks.com/propertyDetails/1-BHK-600-Sq-ft-Multistorey-Apartment-FOR-Sale-Kaithappoyil-in-Kozhikode&id=4d423531383737353431
Kozhikode: 1 BHK Flat for Sale in East Hill Society, East Hill, Kozhikode | Price: ₹17 Lac | Furnishing: Furnished | Link: https://www.magicbricks.com/propertyDetails/1-BHK-500-Sq-ft-Multistorey-Apartment-FOR-Sale-East-Hill-in-Kozhikode&id=4d423631343832353437
Kozhikode: 1 BHK Flat for Sale in Landmark World, Thondayad, Kozhikode | Price: ₹44 Lac | Furnishing: Furnished | Link: https://www.magicbricks.com/propertyDetails/1-BHK-800-Sq-ft-Multistorey-Apartment-FOR-Sale-Thondayad-in-Kozhikode&id=4d423735363336313233
Kozhikode: 1 BHK Flat for Sale in West Hill, Kozhikode | Price: ₹23 Lac | Furnishing: Furnished | Link: https://www.magicbricks.com/propertyDetails/1-BHK-650-Sq-ft-Builde